In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf

data preprocessing
1. dataset 불러오기
2. Encoding categorical data (gender)
3. one hot encoding at '국가' column
4. training set 과 test set 분리하기
5. feature scaling

In [35]:
dataset = pd.read_csv('./data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


Encoding categorical data (gender)

In [36]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


one hot encoding at '국가' column

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


training, test set 분할

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

feature Scaling

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ANN 구축하기
1. ANN 초기화하기
2. input layer와 첫번째 hidden layer 추가하기
3. 두번째 hidden layer 추가하기
4. output layer 추가하기

ANN 초기화하기

In [40]:
ann = tf.keras.models.Sequential()

input layer와 첫번째 hidden layer 추가하기

In [41]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))  # units는 뉴런의 수와 일치해야 한다. 어떻게 정할까? 실험을 바탕으로 한다.
                                                            # 관련이 없거나 과해보이는 수는 안된다. (결국 그냥 개발자가 원하는, 단 터무니없지 않은 수로 한다.)

두번째 hidden layer 추가하기

In [42]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))  # 두번째 hidden layer는 첫번째 hidden layer와 동일하게 해준다.
                                                            # units의 수를 바꿔도 상관 없다 더 정확한 accuracy를 얻을 수 있을지도 모른다.

output layer 추가하기

In [43]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))  # output layer는 units=1 (출력값이 0, 1인 이진변수 이니까 2의0승으로 즉 1개의 뉴런만 있으면 충분하다.
                                                               # 만약 ABC 3개의 출력이 나온다면 onehot encoding을 하게 되었을 때에는 100 010 001 을 나타낼 수 있게 3개의 뉴런이 필요하다.
                                                               # 활성화함수는 sigmoid (요즘은 softmax)를 사용한다.

ANN training 하기
1. compiling ANN
2. training set으로 ANN training 하기

compiling ANN using optimizer, loss func, metrics

In [44]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

training set으로 ANN training 하기

In [45]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 25)

Epoch 1/25
250/250 [==============================] - 1s 4ms/step - loss: 0.6563 - accuracy: 0.6599
Epoch 2/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4877 - accuracy: 0.7962
Epoch 3/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4453 - accuracy: 0.8071
Epoch 4/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4312 - accuracy: 0.8087
Epoch 5/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4203 - accuracy: 0.8124
Epoch 6/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4082 - accuracy: 0.8214
Epoch 7/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3940 - accuracy: 0.8304
Epoch 8/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3814 - accuracy: 0.8371
Epoch 9/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3724 - accuracy: 0.8444
Epoch 10/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3654 - accuracy: 0.8475

In [46]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.04260495]]


In [47]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [48]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1504   91]
 [ 198  207]]


0.8555